## 시설물 점검진단이력 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "시설물 점검진단이력 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/시설물 점검진단이력 목록/selectIoFmChckDinsHstList.csv'

In [2]:
# 개별 시설물 점검진단계획목록 수
g_totalCount = 0

In [3]:
##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
SERVICENAME

'selectIoFmChckDinsHstList'

In [4]:
# REQPARAM = targetData["요청변수"].values[0]
# REQPARAM = REQPARAM.split(",")
# PRIMARYKEY = targetData["기본키"].values[0]
# REQPARAM

In [5]:
JSONKEY = "detail1"
DUMMY = 0

In [6]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/시설물 목록/selectIoFmMngList.csv", encoding="ms949")

In [7]:
dfCombi = imsiDf.loc[:,["fcno"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

7913

## 시설물 점검진단계획 목록 전용 스크랩 함수

In [8]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal"):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata == "NODATA_ERROR"):
                        print("{} page is empty".format(i))
                        break
                    elif( jsondata == "DB_ERROR"):
                        print("DB ERROR")
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 detailList1~5까지 봐야 한다.
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)
                # fcno를 키로 기본정보와 시설물정보를 merge하기 위함
                print("fcno ",emptyPd['fcno'].values[0])
                fcno = emptyPd['fcno'].values[0]
                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount == 0:
                        g_totalCount = int(response.json()["response"]["body"]["totalCount"])
                    jsondata = response.json()["response"]["body"]["items"]
                    if( jsondata == []):
                        print("items is empty")
                    else:
#                         jsondata["index"]=[0]
                        rowData = pd.DataFrame(jsondata)
                        
                        rowData['fcno'] = fcno # 시설물 정보에 fcno칼럼을 수동으로 추가 (머지키로 활용하기 위함)
                        #emptyPd = pd.concat([emptyPd,rowData],axis=1).reindex(emptyPd.index)
                        emptyPd = pd.merge(left = emptyPd , right = rowData, on = "fcno")
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [10]:

numOfRows = 1000
PAGEYN=0

In [ ]:
resultDfMerged = pd.DataFrame()
resultDf = pd.DataFrame()

for i in range(0,dfCombiLen):
    fcnoValue = dfCombi.loc[i].fcno
    pageNo = 1
    global g_totalCount
    g_totalCount = 0
    print("i=",i," fcnoValue= ",fcnoValue)
    while True:
        BASEPARAM={"serviceKey":APIKEY,"pageNo": pageNo, 'fcno':fcnoValue, 'numOfRows':numOfRows, "type":"json"}
        resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY)
        if resultDf.empty: # 정상 데이터가 없는 경우
            break
        else:
            resultDfMerged = resultDfMerged.append(resultDf) 
            g_totalCount -= numOfRows
            print("g_totalCount : ", g_totalCount, " numOfRows : ", numOfRows)
            if(g_totalCount <= 0):
                break
            pageNo += 1
        

        

i= 0  fcnoValue=  017F2520A338CB0F98899109C3F1E127
1 page scraping start
fcno  017F2520A338CB0F98899109C3F1E127
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 1  fcnoValue=  017F25212C92C63041C749D8850C27F0
1 page scraping start
fcno  017F25212C92C63041C749D8850C27F0
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 2  fcnoValue=  017F2521A0A3E80974D725F1B6DB68EF
1 page scraping start
fcno  017F2521A0A3E80974D725F1B6DB68EF
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 3  fcnoValue=  017F25224310A850FB962985F3EC1D60
1 page scraping start
fcno  017F25224310A850FB962985F3EC1D60
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 4  fcnoValue=  017F2524A82A3053A24976923A690EA9
1 page scraping start
fcno  017F2524A82A3053A24976923A690EA9
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 5  fcnoValue=  017F252518748A0FACF1C149F048DDE7
1 page scraping start
fcno  017F252518748A0FACF1C149F0

fcno  017B5BE646FCD2F75FCD5ED0AE53AB04
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 47  fcnoValue=  017B5BE6B96501F8236A3BC70C8FFAA6
1 page scraping start
fcno  017B5BE6B96501F8236A3BC70C8FFAA6
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 48  fcnoValue=  017B5BE70E0BD65D01FEEE3327B1590F
1 page scraping start
fcno  017B5BE70E0BD65D01FEEE3327B1590F
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 49  fcnoValue=  017B5BE7709C23A7AD99F759F9BB5492
1 page scraping start
fcno  017B5BE7709C23A7AD99F759F9BB5492
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 50  fcnoValue=  017B5BE7C20285F9DF67F8429764001C
1 page scraping start
fcno  017B5BE7C20285F9DF67F8429764001C
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 51  fcnoValue=  017B5BEA02E68B4C0C12D73F54FC829C
1 page scraping start
fcno  017B5BEA02E68B4C0C12D73F54FC829C
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows : 

fcno  0178C986512C07F2DE4C044A24A15F60
>totalCount 2
0 no pageNo
g_totalCount :  -998  numOfRows :  1000
i= 93  fcnoValue=  0177CD93229F60B61CE109D9AA46F983
1 page scraping start
fcno  0177CD93229F60B61CE109D9AA46F983
>totalCount 2
0 no pageNo
g_totalCount :  -998  numOfRows :  1000
i= 94  fcnoValue=  017D7474887EABDE8672959B42FAA6E2
1 page scraping start
fcno  017D7474887EABDE8672959B42FAA6E2
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 95  fcnoValue=  017D74748469C2A89CAA9D29D4F890B3
1 page scraping start
fcno  017D74748469C2A89CAA9D29D4F890B3
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 96  fcnoValue=  017D747489348DCDA8C446EA73892749
1 page scraping start
fcno  017D747489348DCDA8C446EA73892749
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 97  fcnoValue=  017D7474882E65DCC52BF9AD81528B2A
1 page scraping start
fcno  017D7474882E65DCC52BF9AD81528B2A
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1

fcno  0176B23F060B25C79C1C03E30F67CAF7
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 139  fcnoValue=  0176B23F07FA5D7D007CB83B6DED5007
1 page scraping start
fcno  0176B23F07FA5D7D007CB83B6DED5007
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 140  fcnoValue=  0176B26462F8476277D1101C61534381
1 page scraping start
fcno  0176B26462F8476277D1101C61534381
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 141  fcnoValue=  0176B23F07A09C2D4C9702457FF36CAE
1 page scraping start
fcno  0176B23F07A09C2D4C9702457FF36CAE
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 142  fcnoValue=  0176B26464C54303CD83EC0448856D0F
1 page scraping start
fcno  0176B26464C54303CD83EC0448856D0F
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRows :  1000
i= 143  fcnoValue=  0176B23F06DCCF480D66094103909D2C
1 page scraping start
fcno  0176B23F06DCCF480D66094103909D2C
>totalCount 0
0 no pageNo
g_totalCount :  -1000  numOfRo

fcno  016FB12385C8131E5F9112A27769B5E7
>totalCount 4
0 no pageNo
g_totalCount :  -996  numOfRows :  1000
i= 185  fcnoValue=  01701DF6E44FE8E5FB62DAB3FCF9C14A
1 page scraping start
fcno  01701DF6E44FE8E5FB62DAB3FCF9C14A
>totalCount 4
0 no pageNo
g_totalCount :  -996  numOfRows :  1000
i= 186  fcnoValue=  01700DA666D7D524DF209BB80427C7D5
1 page scraping start
fcno  01700DA666D7D524DF209BB80427C7D5
>totalCount 4
0 no pageNo
g_totalCount :  -996  numOfRows :  1000
i= 187  fcnoValue=  01700A1FE976402ACDE96EC21005CF11
1 page scraping start
fcno  01700A1FE976402ACDE96EC21005CF11
>totalCount 4
0 no pageNo
g_totalCount :  -996  numOfRows :  1000
i= 188  fcnoValue=  017009E712159E6A8FF59653E82F5C80
1 page scraping start
fcno  017009E712159E6A8FF59653E82F5C80
>totalCount 4
0 no pageNo
g_totalCount :  -996  numOfRows :  1000
i= 189  fcnoValue=  017009DD795C803EA7680048410746BA
1 page scraping start
fcno  017009DD795C803EA7680048410746BA
>totalCount 4
0 no pageNo
g_totalCount :  -996  numOfRows :  

...

fcno  E673B64273DC1C52E034080020A9316A
>totalCount 41
0 no pageNo
g_totalCount :  -959  numOfRows :  1000
i= 6361  fcnoValue=  BEE3CFCF5C9C4586A47CF9D35D7AF4C0
1 page scraping start
fcno  BEE3CFCF5C9C4586A47CF9D35D7AF4C0
>totalCount 128
0 no pageNo
g_totalCount :  -872  numOfRows :  1000
i= 6362  fcnoValue=  E673B64273D21C52E034080020A9316A
1 page scraping start
fcno  E673B64273D21C52E034080020A9316A
>totalCount 98
0 no pageNo
g_totalCount :  -902  numOfRows :  1000
i= 6363  fcnoValue=  E673B64273D11C52E034080020A9316A
1 page scraping start
fcno  E673B64273D11C52E034080020A9316A
>totalCount 98
0 no pageNo
g_totalCount :  -902  numOfRows :  1000
i= 6364  fcnoValue=  E673B64273D01C52E034080020A9316A
1 page scraping start
fcno  E673B64273D01C52E034080020A9316A
>totalCount 105
0 no pageNo
g_totalCount :  -895  numOfRows :  1000
i= 6365  fcnoValue=  E673B64273CF1C52E034080020A9316A
1 page scraping start
fcno  E673B64273CF1C52E034080020A9316A
>totalCount 99
0 no pageNo
g_totalCount :  -901  

fcno  E673B64273B81C52E034080020A9316A
>totalCount 68
0 no pageNo
g_totalCount :  -932  numOfRows :  1000
i= 6406  fcnoValue=  E673B64273B91C52E034080020A9316A
1 page scraping start
fcno  E673B64273B91C52E034080020A9316A
>totalCount 68
0 no pageNo
g_totalCount :  -932  numOfRows :  1000
i= 6407  fcnoValue=  E673B64273C21C52E034080020A9316A
1 page scraping start
fcno  E673B64273C21C52E034080020A9316A
>totalCount 76
0 no pageNo
g_totalCount :  -924  numOfRows :  1000
i= 6408  fcnoValue=  E673B64273D61C52E034080020A9316A
1 page scraping start
fcno  E673B64273D61C52E034080020A9316A
>totalCount 75
0 no pageNo
g_totalCount :  -925  numOfRows :  1000
i= 6409  fcnoValue=  E673B64273C11C52E034080020A9316A
1 page scraping start
fcno  E673B64273C11C52E034080020A9316A
>totalCount 75
0 no pageNo
g_totalCount :  -925  numOfRows :  1000
i= 6410  fcnoValue=  5CFF67FDC7104267E054B099289CEB28
1 page scraping start
fcno  5CFF67FDC7104267E054B099289CEB28
>totalCount 8
0 no pageNo
g_totalCount :  -992  num

fcno  E673B642733D1C52E034080020A9316A
>totalCount 93
0 no pageNo
g_totalCount :  -907  numOfRows :  1000
i= 6451  fcnoValue=  E673B64272FE1C52E034080020A9316A
1 page scraping start
fcno  E673B64272FE1C52E034080020A9316A
>totalCount 70
0 no pageNo
g_totalCount :  -930  numOfRows :  1000
i= 6452  fcnoValue=  E673B64273061C52E034080020A9316A
1 page scraping start
fcno  E673B64273061C52E034080020A9316A
>totalCount 69
0 no pageNo
g_totalCount :  -931  numOfRows :  1000
i= 6453  fcnoValue=  E673B64273011C52E034080020A9316A
1 page scraping start
fcno  E673B64273011C52E034080020A9316A
>totalCount 70
0 no pageNo
g_totalCount :  -930  numOfRows :  1000
i= 6454  fcnoValue=  E673B64273041C52E034080020A9316A
1 page scraping start
fcno  E673B64273041C52E034080020A9316A
>totalCount 69
0 no pageNo
g_totalCount :  -931  numOfRows :  1000
i= 6455  fcnoValue=  E673B64273031C52E034080020A9316A
1 page scraping start
fcno  E673B64273031C52E034080020A9316A
>totalCount 69
0 no pageNo
g_totalCount :  -931  nu

fcno  E673B64273331C52E034080020A9316A
>totalCount 38
0 no pageNo
g_totalCount :  -962  numOfRows :  1000
i= 6496  fcnoValue=  E673B64273321C52E034080020A9316A
1 page scraping start
fcno  E673B64273321C52E034080020A9316A
>totalCount 44
0 no pageNo
g_totalCount :  -956  numOfRows :  1000
i= 6497  fcnoValue=  E673B64272ED1C52E034080020A9316A
1 page scraping start
fcno  E673B64272ED1C52E034080020A9316A
>totalCount 69
0 no pageNo
g_totalCount :  -931  numOfRows :  1000
i= 6498  fcnoValue=  E673B64273401C52E034080020A9316A
1 page scraping start
fcno  E673B64273401C52E034080020A9316A
>totalCount 48
0 no pageNo
g_totalCount :  -952  numOfRows :  1000
i= 6499  fcnoValue=  E673B642733F1C52E034080020A9316A
1 page scraping start
fcno  E673B642733F1C52E034080020A9316A
>totalCount 49
0 no pageNo
g_totalCount :  -951  numOfRows :  1000
i= 6500  fcnoValue=  E673B64272F21C52E034080020A9316A
1 page scraping start
fcno  E673B64272F21C52E034080020A9316A
>totalCount 68
0 no pageNo
g_totalCount :  -932  nu

fcno  E673B64273A71C52E034080020A9316A
>totalCount 47
0 no pageNo
g_totalCount :  -953  numOfRows :  1000
i= 6541  fcnoValue=  56AE6CED62134dcd86596320B2C7267D
1 page scraping start
fcno  56AE6CED62134dcd86596320B2C7267D
>totalCount 66
0 no pageNo
g_totalCount :  -934  numOfRows :  1000
i= 6542  fcnoValue=  A808DC5801294f4d95514056FF9A5E96
1 page scraping start
fcno  A808DC5801294f4d95514056FF9A5E96
>totalCount 64
0 no pageNo
g_totalCount :  -936  numOfRows :  1000
i= 6543  fcnoValue=  6987BC66C1E3418198594FA8A7D8BC18
1 page scraping start
fcno  6987BC66C1E3418198594FA8A7D8BC18
>totalCount 66
0 no pageNo
g_totalCount :  -934  numOfRows :  1000
i= 6544  fcnoValue=  D51D400DCED940ae862775FE7A6BE904
1 page scraping start
fcno  D51D400DCED940ae862775FE7A6BE904
>totalCount 66
0 no pageNo
g_totalCount :  -934  numOfRows :  1000
i= 6545  fcnoValue=  87F8A689B9634598A897816A58DB6A0C
1 page scraping start
fcno  87F8A689B9634598A897816A58DB6A0C
>totalCount 63
0 no pageNo
g_totalCount :  -937  nu

In [83]:
resultDfMerged.shape

(50, 26)

In [84]:
cf.savedata(resultDfMerged, SITENAME,DATANAME,SERVICENAME)

시설물 점검진단계획 목록 save compled


In [85]:
resultDfMerged.shape

(50, 26)